In [1]:
import pandas as pd
import requests
from pandas_profiling import ProfileReport
import sweetviz as sv
import numpy as np
from scipy.stats import chi2_contingency

In [2]:
data_raw = pd.read_csv('./data/NCDB_1999_to_2014.csv',low_memory=False)

In [3]:
data_raw.C_SEV.value_counts()

2    5761772
1      98633
Name: C_SEV, dtype: int64

In [4]:
data_raw.columns

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_SEV', 'C_VEHS', 'C_CONF',
       'C_RCFG', 'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE',
       'V_YEAR', 'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_ISEV', 'P_SAFE',
       'P_USER'],
      dtype='object')

In [6]:
target = 'P_ISEV'
collision_columns = [col for col in data_raw.columns if col[0]=="C"]
people_columns = [col for col in data_raw.columns if col[0]=="P"]
vehicle_columns = [col for col in data_raw.columns if col[0]=="V"]
unnecessary_columns = ['P_ID','V_ID','C_YEAR','V_YEAR']
if target == 'P_ISEV':
    unnecessary_columns.append('C_SEV')
if target == 'C_SEV':
    unnecessary_columns.append('P_ISEV')
int_types = ["P_AGE","V_AGE","C_VEHS"]

In [7]:
data_raw.dtypes.to_dict()

{'C_YEAR': dtype('int64'),
 'C_MNTH': dtype('O'),
 'C_WDAY': dtype('O'),
 'C_HOUR': dtype('O'),
 'C_SEV': dtype('int64'),
 'C_VEHS': dtype('O'),
 'C_CONF': dtype('O'),
 'C_RCFG': dtype('O'),
 'C_WTHR': dtype('O'),
 'C_RSUR': dtype('O'),
 'C_RALN': dtype('O'),
 'C_TRAF': dtype('O'),
 'V_ID': dtype('O'),
 'V_TYPE': dtype('O'),
 'V_YEAR': dtype('O'),
 'P_ID': dtype('O'),
 'P_SEX': dtype('O'),
 'P_AGE': dtype('O'),
 'P_PSN': dtype('O'),
 'P_ISEV': dtype('O'),
 'P_SAFE': dtype('O'),
 'P_USER': dtype('O')}

In [15]:
#New columns
crash_df = data_raw.copy()
crash_df = crash_df[[col for col in collision_columns+people_columns+vehicle_columns if col not in unnecessary_columns]]
crash_df['V_AGE'] = data_raw['C_YEAR'] - pd.to_numeric(data_raw['V_YEAR'], errors='coerce')
categories = [col for col in crash_df.columns if col not in int_types and col != target]

In [16]:
crash_df.head()

,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,V_TYPE,V_AGE
0,01,1,20,02,34,UU,1,5,3,03,M,41,11,1,UU,1,06,9.0
1,01,1,20,02,34,UU,1,5,3,03,M,19,11,1,UU,1,01,12.0
2,01,1,20,02,34,UU,1,5,3,03,F,20,13,2,02,2,01,12.0
3,01,1,08,01,01,UU,5,3,6,18,M,46,11,1,UU,1,01,13.0
4,01,1,08,01,01,UU,5,3,6,18,M,05,99,2,UU,3,NN,NaN


In [17]:
def replace_categories_nas(data,categories):
    nas = ["UU","XX","U","X","XXXX","NN","NNNN","N","UUUU"]
    others = {
        "P_SAFE" : "14",
        "P_PSN": "100",
        "V_TYPE": "24",
        "C_TRAF": "19",
        "C_RALN": "7",
        "C_RSUR": "10",
        "C_WTHR": "8",
        "C_RCFG": "13",
        "C_CONF": "42"
    }
    data[categories] = data[categories].replace(to_replace=rf'^({"|".join(nas)})',value=pd.NA,regex=True)
    for key,value in others.items():
        data[key] = data[key].replace(to_replace=r'^Q.*', value=value, regex=True)

def transform_hour(value):
    value = pd.to_numeric(value,errors='coerce')
    if value >= 0 and value<6:
        return '0'
    if value >= 6 and value<12:
        return '1'
    if value >= 12 and value<18:
        return '2'
    if value >= 18 and value<24:
        return '3'
    return pd.NA

In [18]:
replace_categories_nas(crash_df,[col for col in crash_df.columns if col not in int_types])
crash_df['C_HOUR'] = crash_df['C_HOUR'].apply(transform_hour) 
crash_df[int_types] = crash_df[int_types].apply(pd.to_numeric, errors='coerce',downcast='integer')

In [19]:
crash_df = crash_df.drop_duplicates()

In [20]:
print(f'Numero de registros: {crash_df.shape[0]}')
print(f'Numero de columnas: {crash_df.shape[1]}')

Numero de registros: 5736742
Numero de columnas: 18


In [21]:
crash_df[categories].astype('category').dtypes.to_dict()

{'C_MNTH': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12'],
                  ordered=False),
 'C_WDAY': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7'], ordered=False),
 'C_HOUR': CategoricalDtype(categories=['0', '1', '2', '3'], ordered=False),
 'C_CONF': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '21', '22', '23', '24',
                   '25', '31', '32', '33', '34', '35', '36', '41', '42'],
                  ordered=False),
 'C_RCFG': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '13'],
                  ordered=False),
 'C_WTHR': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7', '8'], ordered=False),
 'C_RSUR': CategoricalDtype(categories=['1', '10', '2', '3', '4', '5', '6', '7', '8', '9'], ordered=False),
 'C_RALN': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7'], ordered=False),


In [22]:
crash_df.head()

,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,V_TYPE,V_AGE
0,01,1,3,2.0,34,<NA>,1,5,3,03,M,41.0,11,1,<NA>,1,06,9.0
1,01,1,3,2.0,34,<NA>,1,5,3,03,M,19.0,11,1,<NA>,1,01,12.0
2,01,1,3,2.0,34,<NA>,1,5,3,03,F,20.0,13,2,02,2,01,12.0
3,01,1,1,1.0,01,<NA>,5,3,6,18,M,46.0,11,1,<NA>,1,01,13.0
4,01,1,1,1.0,01,<NA>,5,3,6,18,M,5.0,99,2,<NA>,3,<NA>,NaN


In [23]:
crash_df = crash_df.dropna(subset=[target])
crash_df[target] = crash_df[target].astype('int8')
crash_df[target] = crash_df[target].replace({1:0,2:0,3:1})

In [24]:
df_dtypes = pd.merge(crash_df.isnull().sum(axis = 0).sort_values().to_frame('missing_value').reset_index(),
         crash_df.dtypes.to_frame('feature_type').reset_index(),
         on = 'index',
         how = 'inner')
df_dtypes

,index,missing_value,feature_type
0,P_ISEV,0,int64
1,C_MNTH,218,object
2,C_VEHS,333,float64
3,C_WDAY,1139,object
4,P_PSN,10682,object
5,C_HOUR,54459,object
6,C_RSUR,70748,object
7,C_WTHR,76621,object
8,P_USER,84995,object
9,P_SEX,89061,object


In [25]:
print(f'Registros Sin muertes {crash_df[crash_df[target]==0][target].count()}')
print(f'Registros Con muertes {crash_df[crash_df[target]==1][target].count()}')

Registros Sin muertes 5347197
Registros Con muertes 40150


In [37]:
profile = ProfileReport(crash_df, minimal=True)
profile.to_file("./eda_output/output_pisev.html")

In [38]:
crash_df.to_csv('./data/crash_transformed_p_isev.csv',index=False)